In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyter_helpers.notifications import Notifications

#help(Notifications)

p = '/usr/share/sounds/gnome/default/alerts/'
p1 = p + 'glass.ogg'
p2 = p + 'sonar.ogg'
Notifications(success_audio=p1, time_threshold=2,
    failure_audio=p2) 
#    ,integration='GNOME')

#from jupyter_helpers.notifications import NotificationsNotifications(

In [ ]:
#fast until the end
#%debug
%load_ext autoreload
%autoreload 2

#--plot_types raw_stats_scatter,feat_stats_scatter\
import warnings

#%debug
#with warnings.catch_warnings():
#warnings.filterwarnings('error')
%run -i ../run/run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_hold \
 --feat_types Hjorth\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcorr 0\
 --save_rbcorr 0\
 --load_rbcorr 0\
 --use_existing_TFR 0\
 --load_only 0\
 --show_plots 0\
 --scale_data_combine_type subj \
 --rbcorr_use_local_means 1\
 --output_subdir test\
 --stats_fn_prefix stats_S99_1_ \
 --brain_side_to_use both \
 --param_file genfeats_defparams.ini




In [ ]:
new_main_body_side

In [ ]:
np.append(names_lfp, names_src)

In [ ]:
feature_names_all

In [ ]:
feature_names_all[-20:]

In [ ]:
subfeature_order

In [ ]:
chnames_tfr

In [ ]:
#fast until the end
#%debug
%load_ext autoreload
%autoreload 2

#--plot_types raw_stats_scatter,feat_stats_scatter\

%run -i ../run/run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types bpcorr,Hjorth,con,rbcorr\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcorr 0\
 --save_rbcorr 0\
 --load_rbcorr 0\
 --use_existing_TFR 0\
 --load_only 0\
 --show_plots 0\
 --scale_data_combine_type subj \
 --rbcorr_use_local_means 1\
 --output_subdir test\
 --stats_fn_prefix stats_S97,S99_3_ \
 --param_file genfeats_defparams.ini

In [ ]:
# check whetehr do side filtering / reversal
loadRaws
collectdataFromMultiRaws
collectAllMarkedIntervals
gatherStats
gatharMultiBandStats

In [ ]:
mne.cuda.init_cuda()

In [ ]:
gv.CUDA_state

In [ ]:
n_jobs

In [ ]:
if bands_only == 'fine':
    # no need to dubplicate reversing order
    bandPairs = [('tremor','low_beta', 'corr'), ('tremor','high_beta', 'corr'),
                ('tremor','low_gamma', 'corr'), ('tremor','high_gamma', 'corr'),
                ('low_beta','low_gamma', 'corr') , ('low_beta','high_gamma', 'corr') ,
                ('high_beta','low_gamma', 'corr') , ('high_beta','high_gamma', 'corr') ]
else:
    bandPairs = [('tremor','beta', 'corr'), ('tremor','gamma', 'corr'), ('beta','gamma', 'corr') ]
if use_lfp_HFO:
    # HFO should be always on the second place unless we use HFO-HFO
    # coupling in LFP!
    bandPairs += [ ('tremor','HFO', 'corr'), ('beta','HFO', 'corr'), ('gamma','HFO', 'corr') ]

    if bands_only == 'fine':
        bandPairs += [ ('HFO1','HFO2', 'div'), ('HFO1','HFO3', 'div'), ('HFO2','HFO3', 'div') ]

bpcorrs_pri     = [0] * len(rawnames)
wbd_bpcorr_pri  = [0] * len(rawnames)
bpcor_names_pri = [0] * len(rawnames)
for rawind in range(len(dat_pri)):
    fn = pre_bpcorr_fname.format(rawnames[rawind], n_channels_pri[rawind] )
    fn = pjoin(gv.data_dir, output_subdir, fn)
    if load_bpcorr and os.path.exists(fn):
        print('Load bpcorr from {}'.format(fn) )
        f = np.load(fn, allow_pickle=True)
        bpcorrs      =f['bpcorrs'    ]
        bpcor_names  =f['bpcor_names']
        wbd_bpcorr   =f['wbd_bpcorr' ]
    else:
        print('Starting bpcorr for datset {}'.format(rawind) )

        raw_perband =  raw_perband_bp_pri[rawind]
        chnames_perband =  chnames_perband_bp_pri[rawind]
        means_perband = means_perband_bp_pri[rawind]
        bpcorrs = []
        bpcor_names = []
        for bp in bandPairs:
            print('band pair ',bp)
            bpcorrs_curbp,bpcor_names_curbp,dct_nums,wbd_bpcorr =\
            computeCorr(raw_perband, names=chnames_perband,
                                    defnames=chnames_tfr,
                                    parcel_couplings=parcel_couplings,
                                    LFP2parcel_couplings=LFP2parcel_couplings,
                                    LFP2LFP_couplings=LFP2LFP_couplings,
                                    res_group_id=newchn_grouping_ind,
                                    skip=skip, windowsz = windowsz,
                                    band_pairs = [bp], n_jobs=n_jobs,
                                    positive=1, templ='{}_.*',
                                    sort_keys=srcgrouping_names_sorted,
                                    means=means_perband)
            gc.collect()
            assert len(bpcor_names_curbp) > 0
            bpcorrs += bpcorrs_curbp
            bpcor_names += bpcor_names_curbp

        bpcorrs = np.vstack(bpcorrs)

        for feati in range(len(bpcor_names) ):
            bpcor_names[feati] = 'bp' + bpcor_names[feati]


        if save_bpcorr:
            print('Saving bpcorr ',fn)
            np.savez( fn , bpcorrs=bpcorrs, bpcor_names=bpcor_names, wbd_bpcorr=wbd_bpcorr)
            gc.collect()

    bpcor_names_pri [rawind] = bpcor_names
    bpcorrs_pri     [rawind] = bpcorrs
    wbd_bpcorr_pri  [rawind] = wbd_bpcorr

if do_cleanup:
    del  bpcorrs
    del  raw_perband_bp_pri
gc.collect()


In [ ]:
# [rawind] [bandname]

In [ ]:
# means com gatherMultiBandStats

# BPCORR

In [ ]:
means_perband_bp_pri_,_ = \
ugf.gatherMultiBandStats(rawnames,raw_perband_bp_pri, times_pri,
                        chnames_perband_bp_pri, side_switched_pri, sfreq,
                        baseline_int, scale_data_combine_type,
                        artif_handling)

    means_perband_bp_pri = len(rawnames) * [dict()]
    for rawind in range(len(rawnames)):
        for bandname in means_perband_bp_pri_[rawind]:
            means_perband_bp_pri[rawind][bandname] = \
                means_perband_bp_pri_[rawind][bandname][baseline_int]


In [ ]:
%load_ext autoreload
%autoreload 2
import utils_genfeats as ugf
import utils
import mne

In [ ]:
dat1 = np.ones((2,512))
dat2 = np.ones((2,512))
dat2[:,256:] = 0.5
raw1 = utils.makeSimpleRaw(dat1,rescale=0)
raw2 = utils.makeSimpleRaw(dat2,rescale=0)
raw1.get_data()

In [ ]:
%precision %.4f
import numpy as np
np.set_printoptions(precision=4, suppress=1)

In [ ]:
import utils_genfeats as ugf
import utils
import gc
#%debug




def test2ch(bandPairs, ch1_band1,ch2_band1,ch1_band2,ch2_band2, chn1,chn2, means_perband, windowsz, skip):

    # ch1_band1 = np.ones(dat_len)
    # #ch2_band1 = np.ones(dat_len)
    # ch2_band1 = np.random.uniform(size=dat_len)
    dat1 = np.array( [ch1_band1, ch2_band1] )

    # ch1_band2 = np.ones(dat_len)
    # ch2_band2 = np.ones(dat_len)
    dat2 = np.array( [ch1_band2, ch2_band2] )

    raw1 = utils.makeSimpleRaw(dat1,rescale=0)
    raw2 = utils.makeSimpleRaw(dat2,rescale=0)
    raw_perband = {}
    raw_perband[bp[0]]   = raw1
    raw_perband[bp[1]]   = raw2

    

    chnames_perband = {}
    # I cannot put just one band, it should have same bands in both modalities
    # unless the band is LFP
    # chnames_perband is one list for all modalities
    chnames_perband[bp[0]] = [chn1,chn2]
    chnames_perband[bp[1]] = [chn1,chn2]  # can be L as well if CB
    
    
    
    for i in range(2):
        display(bp[i], raw_perband[bp[i]].get_data())
    display(chnames_perband)
    display(means_perband)


    srcgrouping_names_sorted = []

    # I think I want LFP to go first
    chnames_tfr = [chn1,chn2]
    print('chnames_tfr = ',chnames_tfr)

    parcel_couplings = []
    LFP2parcel_couplings = []
    LFP2LFP_couplings = []
    
    from utils import parseMEGsrcChnameShort
    
    if chn1.startswith('LFP') and chn2.startswith('msrc'):
        side2, gi2, parcel_ind2, si2  = parseMEGsrcChnameShort(chn2)
        LFP2parcel_couplings = { (chn1, parcel_ind2): [(0,1)]  }
        display(LFP2parcel_couplings)
    if chn1.startswith('msrc') and chn2.startswith('msrc'):        
        side1, gi1, parcel_ind1, si1  = parseMEGsrcChnameShort(chn1)
        side2, gi2, parcel_ind2, si2  = parseMEGsrcChnameShort(chn2)
        # keys: tuple of parcel indices,  values: chnames_tfr indices of chnames
        parcel_couplings = { (parcel_ind1, parcel_ind2):[(0,1)] }
        display(parcel_couplings)
    if chn1.startswith('LFP') and chn2.startswith('LFP'):
        LFP2LFP_couplings = { (chn1,chn2):[(0,1)] }
        display(LFP2LFP_couplings)

    n_jobs = 1
    newchn_grouping_ind = 10
    

    # test differen simple imput combinations, see that correl indeed correls

    bpcorrs = []
    bpcor_names = []
    for bpcur in bandPairs:
        print('band pair ',bpcur)
        bpcorrs_curbp,bpcor_names_curbp,dct_nums,wbd_bpcorr =\
        ugf.computeCorr(raw_perband, names=chnames_perband,
                                defnames=chnames_tfr,
                                parcel_couplings=parcel_couplings,
                                LFP2parcel_couplings=LFP2parcel_couplings,
                                LFP2LFP_couplings=LFP2LFP_couplings,
                                res_group_id=newchn_grouping_ind,
                                skip=skip, windowsz = windowsz,
                                band_pairs = [bpcur], n_jobs=n_jobs,
                                positive=1, templ='{}_.*',
                                sort_keys=srcgrouping_names_sorted,
                                means=means_perband, reverse=1, verbose=3, pad=0)
        gc.collect()
        assert len(bpcor_names_curbp) > 0
        bpcorrs += bpcorrs_curbp
        bpcor_names += bpcor_names_curbp

    bpcorrs = np.vstack(bpcorrs)

    for feati in range(len(bpcor_names) ):
        bpcor_names[feati] = 'bp' + bpcor_names[feati]

    print('----    Result is')
    display(bpcorrs)
    display(bpcor_names_curbp)
    display(wbd_bpcorr)
    display(dct_nums)
    
    return bpcorrs,bpcor_names_curbp,wbd_bpcorr,dct_nums
    
    
bp = 'tremor','beta'
bandPairs = [(bp[0],bp[1], 'corr')]

windowsz = 4
skip = 2
dat_len = windowsz*5
    
# ones will live in tremor band of channel 1
ch1_band1 = np.ones(dat_len) * 1
ch1_band1[windowsz:] = 0
# twos will live in tremor band of channel 2
ch2_band1 = np.ones(dat_len) * 2
ch2_band1[windowsz*2:] = 0
#ch2_band1 = np.random.uniform(size=dat_len)
# tens will live in beta band of channel 1
ch1_band2 = np.ones(dat_len) * 10
ch1_band2[windowsz*3:] = 0
#ch1_band2[windowsz:] = 0
# twentys will live in beta band of channel 2
ch2_band2 = np.ones(dat_len) * 20
ch2_band2[windowsz*4:] = 0

means_perband = {}
# means_perband[bp[0]] = [0.01, 0.02]
# means_perband[bp[1]] = [0.1, 0.2]
means_perband[bp[0]] = [0.0, 0.0]
means_perband[bp[1]] = [0.0, 0.0]

parcel_index1 = 1
parcel_index2 = 2

chn1 = 'LFPR01'
#chn1 = f'msrcR_9_{parcel_index1}_c1'
#chn2 = f'msrcR_9_{parcel_index2}_c2'
chn2 = 'LFPR02'


res = test2ch(bandPairs,ch1_band1,ch2_band1, ch1_band2, ch2_band2,chn1,chn2, means_perband, windowsz,skip)
bpcorrs,bpcor_names_curbp,wbd_bpcorr,dct_nums = res

In [ ]:
19.8 * .99

In [ ]:
bpcorrs,bpcor_names_curbp,wbd_bpcorr,dct_nums = res

In [ ]:
res = test2ch(bandPairs,np.ones(windowsz),np.ones(windowsz), np.ones(windowsz), 
              np.ones(windowsz),chn1,chn2, windowsz,windowsz)

# Corr and Hjorth together

In [ ]:
oper = 'corr'
pos = True
local_means=False

windowsz = 8
skip = 4
#dfrom = ch1_band1[::8];  dto = ch1_band2[::8];   
dfrom = np.ones(windowsz * 2); dfrom[windowsz:] = 0
dto = np.ones(windowsz * 2)
mfrom = 0; mto = 0
arg = ('resname','bn_from','bn_to','pc','fromi','toi',windowsz,skip,dfrom,dto,mfrom,mto,oper,pos,local_means)
ugf._feat_correl3(arg)

In [ ]:
dfrom, dto

In [ ]:
sto = ugf.stride(dto , win=(windowsz), stepby=(skip,) )
print( sto[0] )
print( sto[1] )
print( sto[-1] )

In [ ]:
sfrom = ugf.stride(dfrom , win=(windowsz), stepby=(skip,) )
print( sfrom[0] )
print( sfrom[1] )
print( sfrom[-1] )

In [ ]:
prod = sfrom * sto
display(prod)
np.mean(prod, axis=-1)

In [ ]:
sv = ugf.stride(np.arange(len(dfrom) )[None,:], win=(1,windowsz), stepby=(1,skip) )
print(sv.shape)
sv

In [ ]:
np.array( [4,5] ) == np.array([4,5])

In [ ]:
U = np.vstack([np.arange(6 ),10+np.arange(6) ] )
sv = ugf.stride( U , win=(1,3), stepby=(1,2) )
print(sv.shape)
sv

In [ ]:
U = np.vstack([np.arange(len(dfrom) ),np.arange(len(dfrom)  ) ] )
sv = ugf.stride( U , win=(1,windowsz), stepby=(1,skip) )
print(sv.shape)
sv

In [ ]:
dfrom.shape,windowsz,skip

In [ ]:
tst = np.zeros(windowsz+skip-1)
ugf.Hjorth(tst[None,:], dt=1, windowsz=windowsz, skip=skip, pad=False)

In [ ]:
tst = np.zeros(windowsz+skip)
ugf.Hjorth(tst[None,:], dt=1, windowsz=windowsz, skip=skip, pad=False)

In [ ]:
#%debug
ugf.Hjorth(dfrom[None,:], dt=1, windowsz=windowsz, skip=skip, pad=False)

# All 

In [ ]:
# ivalis_pri = [{'notrem_L': [(0.0, 3.0, 'notrem_L'),
#    (6.0, 17.0, 'notrem_L'),
#    (23.0, 26.0, 'notrem_L')],
#   'move_L': [(3.0, 6.0, 'move_L')],
#   'trem_L': [(10.0, 20.0, 'trem_L'), (28.0, 32.0, 'trem_L')],
#   'trem_R': [(14.0, 22.7, 'trem_R')],
#   'notrem_R': [(25.0, 29.0, 'notrem_R')]}]

# dat_pri                         = [0]*len(rawnames)
# dat_lfp_hires_pri               = [0]*len(rawnames)
# times_pri                       = [0]*len(rawnames)
# times_hires_pri                 = [0]*len(rawnames)
# subfeature_order_pri            = [0]*len(rawnames)
# subfeature_order_lfp_hires_pri  = [0]*len(rawnames)
# extdat_pri                      = [0]*len(rawnames)
# ivalis_pri                      = [0]*len(rawnames)


In [ ]:
#fast until the end
#%debug
%load_ext autoreload
%autoreload 2

%run -i ../run/run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr\
 --load_only 1\
 --show_plots 0\
 --plot_types ,\
 --scale_data_combine_type subj \
 --output_subdir test\
 --stats_fn_prefix stats_S97,S99_3_ \
 --param_file genfeats_defparams.ini

reclen = 8 # it is not enough because wavelet has len 2048
nbins = windowsz * reclen 
nbins_hires = windowsz_hires * reclen
#########
times_pri = [np.arange(nbins ) * 1/sfreq]
times_hires_pri = [np.arange( nbins_hires) * 1/sfreq_hires]
extdat_pri = [np.zeros( (2,nbins))]
subfeature_order_pri = [['LFPR01', 
 'LFPR12',
 'msrcR_0_2_c5',
 'msrcR_0_18_c15',
 'msrcL_0_19_c1',
 'msrcR_0_3_c8',
 'msrcL_0_60_c24']]

subfeature_order_lfp_hires_pri = [ ['LFPR01', 'LFPR12'] ]
ivalis_pri = [{'notrem_L':(0.5,1,'notrem_L'),'trem_L':(1,1.5,'notrem_L') }]

dat_pri = [np.zeros( (len(subfeature_order_pri[0]), nbins) )]
dat_lfp_hires_pri = [np.zeros( (len(subfeature_order_lfp_hires_pri[0]), nbins_hires) )]        

offset_sec = 4
offset_start = windowsz * offset_sec
for i in range(len(dat_pri[0])):
    dat_pri[0][i][offset_start:] = 1
    
offset_hires_start = windowsz_hires* offset_sec    
for i in range(len(dat_lfp_hires_pri[0])):
    dat_lfp_hires_pri[0][i][offset_hires_start:] = 1

%matplotlib inline
plt.plot(times_pri[0],  dat_pri[0][0])
#plt.figure()
plt.plot(times_hires_pri[0], 1 + dat_lfp_hires_pri[0][0])

In [ ]:
import utils

In [ ]:
reclen = 8 # it is not enough because wavelet has len 2048
nbins = windowsz * reclen 
nbins_hires = windowsz_hires * reclen
#########
times_pri = [np.arange(nbins ) * 1/sfreq]
times_hires_pri = [np.arange( nbins_hires) * 1/sfreq_hires]
extdat_pri = [np.zeros( (2,nbins))]
subfeature_order_pri = [['LFPR01', 
 'LFPR12',
 'msrcR_0_4_c5',
 'msrcR_0_8_c15',
 'msrcR_0_16_c8',
 'msrcR_0_32_c2',
 'msrcR_0_38_c9',
 'msrcR_0_44_c0',
 'msrcR_0_52_c9',
 'msrcR_0_58_c5',
 'msrcL_0_60_c24']]
subfeature_order_lfp_hires_pri = [ ['LFPR01', 'LFPR12'] ]
ivalis_pri = [{'notrem_L':[(0.5,5,'notrem_L')] }] #,'trem_L':[(1,1.5,'trem_L')] }]

rawn = 'S97_off_move'
anndict_per_intcat_per_rawn = {}
for rawn in [rawn]:
    anndict_per_intcat_per_rawn[rawn] = {'artif':{
        'MEG':mne.Annotations([],[],[]), 'LFP': mne.Annotations([],[],[])  },
        'beh_state':[]}
anndict_per_intcat_per_rawn[rawn]['beh_state'] = \
    utils.intervals2anns(ivalis_pri[0]['notrem_L']) # + utils.intervals2anns(ivalis_pri[0]['trem_L'])


dat_pri = [np.zeros( (len(subfeature_order_pri[0]), nbins) )]
dat_lfp_hires_pri = [np.zeros( (len(subfeature_order_lfp_hires_pri[0]), nbins_hires) )]        

offset_sec = 4
offset_start = windowsz * offset_sec
for i in range(len(dat_pri[0])):
    dat_pri[0][i][offset_start:] = 1
    
offset_hires_start = windowsz_hires* offset_sec    
for i in range(len(dat_lfp_hires_pri[0])):
    dat_lfp_hires_pri[0][i][offset_hires_start:] = 1

%matplotlib inline
plt.plot(times_pri[0],  dat_pri[0][0])
#plt.figure()
plt.plot(times_hires_pri[0], 1 + dat_lfp_hires_pri[0][0])

In [ ]:
anndict_per_intcat_per_rawn

In [ ]:
gv.DEBUG_MODE = True
#%debug

# import warnings
# warnings.filterwarnings("error")
# # with warnings.catch_warnings():
# #     warnings.simplefilter('error')
#%debug
# try:
%run -i ../run/run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcorr 0\
 --save_rbcorr 0\
 --load_rbcorr 0\
 --use_existing_TFR 0\
 --use_preloaded_data 1\
 --allow_CUDA 0\
 --load_only 0\
 --show_plots 0\
 --plot_types ,\
 --prescale_data 0\
 --normalize_TFR separate\
 --n_jobs 1\
 --save_feat 0\
 --feat_stats_artif_handling no\
 --scale_data_combine_type subj \
 --rbcorr_use_local_means 0\
 --output_subdir test\
 --stats_fn_prefix stats_S97,S99_3_ \
 --param_file genfeats_defparams.ini 
# except ComplexWarning as e:
#     pass

In [ ]:
feat_dict['con'].shape

In [ ]:
%matplotlib inline
from plots import shadeAnn
anns_artif = anndict_per_intcat_per_rawn[rawnames[0]]['artif']['LFP'] 

N = dat_pri[0].shape[-1]
print(tfrres.shape)
nc = 11
fig,axs = plt.subplots(nc,1,figsize=(12,2*nc), sharex='col')
axind = 0
ax = axs[axind]; axind += 1
for chi in range(2,tfrres.shape[0]):
    for fi in range(8,tfrres.shape[1] ):
        ax.plot( np.abs( tfrres[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] /8 )
#ax.fill_betweenx([0,.2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
shadeAnn(ax,anns_artif, 0,1 ,plot_bins = 1, sfreq=sfreq, skip=skip)

ax.set_title('tfrres SRC higher_freq')
print(tfrres.shape)

ax = axs[axind]; axind += 1
for chi in range(2,tfrres.shape[0]):
    for fi in range(8 ):
        ax.plot( np.abs( tfrres[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] /8 )
ax.set_title('tfrres SRC low_freq')
ax.fill_betweenx([0,2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)

####################

ax = axs[axind]; axind += 1
for feati in range(30,bpow_abscds_all_reshaped.shape[0]):
    ax.plot( np.abs( bpow_abscds_all_reshaped[feati] ) )
ax.plot( dat_pri[0][0][::skip] / 10  )
ax.set_title('bpow_abscds_all_reshaped higher_freq')
ax.fill_betweenx([0,0.2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)

############

# plot LFP
#fig,axs = plt.subplots(2,1,figsize=(12,6))
ax = axs[axind]; axind += 1
for chi in range(2):
    for fi in range(8,tfrres.shape[1] ):
        ax.plot( np.abs( tfrres[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] / 8  )
ax.fill_betweenx([0,.2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.set_title('tfrres LFP higher_freq')

ax = axs[axind]; axind += 1
for chi in range(2):
    for fi in range(8 ):
        ax.plot( np.abs( tfrres[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] *2  )
ax.set_title('tfrres LFP lower_freq')
ax.fill_betweenx([0,2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)

wbdi = (offset_start-windowsz)//skip
s,e = tfrres_wbd_pri[0][:,wbdi] / skip
ax.fill_betweenx([0,1],  s,e, color='yellow', alpha=0.15)

#----

ax = axs[axind]; axind += 1
for chi in range(tfrres_LFP_HFO.shape[0]):
    for fi in range(tfrres_LFP_HFO.shape[1] ):
        ax.plot( np.abs( tfrres_LFP_HFO[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] /8 )
ax.set_title('tfrres LFP HFO')
ax.fill_betweenx([0,0.1],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)


#-------------------
ax = axs[axind]; axind += 1
d1 = act_pri[0][0]
d2 = mob_pri[0][0]
d3 = compl_pri[0][0]
#plt.close('all')
ax.plot( d1 / np.max(d1)  ,label='act', marker='*' )
ax.plot( d2 / np.max(d2), label='mob' )
ax.plot( d3 / np.max(d3)  , label='compl' )


ax.plot( dat_pri[0][0][::skip] *2  )
ax.fill_betweenx([0,2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.set_title('H')

wbdi = (offset_start-windowsz)//skip
s,e = wbd_H_pri[0][:,wbdi] / skip
ax.fill_betweenx([0,1],  s,e, color='yellow', alpha=0.15)
ax.legend()

ax = axs[axind]; axind += 1
for bn,r in raw_perband_flt_pri[0].items():
    d = r.get_data()
    print(bn, np.where( np.abs(d[:,-1] ) > 1e-10 )[0] )
    ax.plot(np.arange(N) / skip, d.T , label=bn)
ax.plot(np.arange(N) / skip, dat_pri[0][0]/2 )        
shadeAnn(ax,anns_artif, np.min(d),np.max(d) ,plot_bins = 1, sfreq=sfreq, skip=skip)
    

ax = axs[axind]; axind += 1
for bn,r in raw_perband_bp_pri[0].items():
    d = r.get_data()
    print(bn, np.where( np.abs(d[:,-1] ) > 1e-10 )[0] )
    ax.plot(np.arange(N) / skip,  d.T , label=bn)
ax.plot( np.arange(N) / skip, dat_pri[0][0]/2 )    


ax = axs[axind]; axind += 1
d = bpcorrs_pri[0][0:5]
ax.plot( d.T, label='bp'  )

d = rbcorrs_pri[0][0:5]
ax.plot( d.T, label='rb'  )
ax.plot( dat_pri[0][0][::skip] *0.02, ls=':'  )

wbdi = (offset_start-windowsz)//skip
s,e = wbd_rbcorr_pri[0][:,wbdi] / skip
ax.fill_betweenx([0,0.02],  s,e, color='yellow', alpha=0.15)

wbdi = (offset_start-windowsz)//skip
s,e = wbd_bpcorr_pri[0][:,wbdi] / skip
ax.fill_betweenx([-0.01,0],  s,e, color='purple', alpha=0.15)

In [ ]:
# raising
# for i in range(len(dat_pri[0])):
#     dat_pri[0][i] = np.arange(nbins) + 2**i * 1e-3
    
# for i in range(len(dat_lfp_hires_pri[0])):
#     dat_lfp_hires_pri[0][i] = np.arange(nbins_hires) + 3**i * 1e-3

In [ ]:
import gc; gc.collect()

plt.close('all')

In [ ]:
main_side

In [ ]:
roi_labels['all_raw'][60]

In [ ]:
    for lbli in range(len(gp.areas_list_aal_my_guess)):
        for lblj in range(lbli+1, len(gp.areas_list_aal_my_guess)):
  
            lab1 = gp.areas_list_aal_my_guess[lbli]
            lab2 = gp.areas_list_aal_my_guess[lblj]
            if exclude_CB_interactions and (lab1.find('Cerebellum') >= 0\
                    or lab2.find('Cerebellum') >= 0):
                continue
                
            op_mainside_let = utils.getOppositeSideStr(main_side_let)
            pc1 = roi_labels['all_raw'].index(lab1 + f'_{op_mainside_let}')
            pc2 = roi_labels['all_raw'].index(lab2 + f'_{op_mainside_let}')
            print(pc1,pc2,lab1,lab2)
            #side = ?
#             templ1 = '.?src_{}.*'.format(lab1)
#             templ2 = '.?src_{}.*'.format(lab2)
            #cross_types += [  (templ1 ,templ2 )  ]

    

In [ ]:
tfrres.shape

In [ ]:
len(subfeature_order)

In [ ]:
d.shape

In [ ]:
%matplotlib inline

In [ ]:
#dd[0].shape, dd_pad[0].shape

In [ ]:
xs = np.arange(dat_pri[0].shape[1])

In [ ]:
dd = ugf.Hjorth(dat_pri[0], dt=1/sfreq, windowsz=windowsz, skip=skip, pad=False)

In [ ]:
plt.close('all')

In [ ]:
tst = np.zeros(windowsz+skip)
dd = ugf.Hjorth(dat_pri[0], dt=1/sfreq, windowsz=windowsz, skip=skip, pad=False)
dd_pad = ugf.Hjorth(dat_pri[0], dt=1/sfreq, windowsz=windowsz, skip=skip, pad=True)
dd_pad_ri = ugf.Hjorth(dat_pri[0], dt=1/sfreq, windowsz=windowsz, skip=skip, pad=True, remove_invalid=True)

In [ ]:
for tuple_ind in range(3):
    plt.figure(figsize=(15,4))
    plt.plot( dd    [tuple_ind][0], label=f'no_pad {len(dd    [tuple_ind][0])}' )
    plt.plot( dd_pad[tuple_ind][0] ,label=f'pad  {len(dd_pad    [tuple_ind][0])}', ls='--'  )
    plt.plot( dd_pad_ri[tuple_ind][0] ,label=f'pad_ri  {len(dd_pad_ri    [tuple_ind][0])}',marker='|',lw=0,c='r' )
    #plt.ylim(-0.01,0.1)
    plt.legend(loc='upper right')
    plt.title(f'tuple_ind = {tuple_ind}')

In [ ]:

# with decim = 1
tfrres_t,wbd_t = utils.tfr(dat_pri[0], sfreq, freqs, n_cycles,
                        windowsz, decim = 1,
                        n_jobs=n_jobs_tfr, mode='valid')

plt.figure()
for chi in range(2):
    for fi in range(27,tfrres_t.shape[1] ):
        plt.plot( np.abs( tfrres_t[chi][fi] ) )
        
nb = tfrres_t[chi][fi].shape[0]
plt.plot(dat_pri[0][0], lw=1)

plt.plot(dat_pri[0][0][nbins - nb:], lw=3, ls=':')
plt.axvline(x=nbins - nb - windowsz/2, ls=':')
plt.axvline(x=nbins - nb + windowsz/2, ls=':')


In [ ]:
bpow_abscds_all_reshaped.shape

In [ ]:
csdord_strs[30:40]

In [ ]:
display( anns_artif )


In [ ]:
for a in anns_artif:
    print(a)


In [ ]:
cmap = plt.cm.get_cmap('tab20', 20)
cmap(3),cmap(10),cmap(3.4),cmap(20),cmap(21)

In [ ]:


ax = plt.gca()
shadeAnn(ax,anns_artif, 0,1 ,plot_bins = 1, sfreq=sfreq, skip=skip)
ax.legend()

In [ ]:
ax.plot( dat_pri[0][0][::skip] / 10  )

In [ ]:
wbd_H_pri[0].shape

In [ ]:
wbd_bpcorr_pri[0].shape

In [ ]:
rbcorrs.shape

In [ ]:
csd_cur.shape, tfrres.shape

In [ ]:
parcel_couplings, LFP2LFP_couplings

In [ ]:
parcel_couplings_t = {(4, 4): [(2, 2)] }
#parcel_couplings_t = {}
LFP2LFP_couplings_t = {('LFPR01', 'LFPR01'): [(0, 0)] }
LFP2parcel_couplings_t = {}

In [ ]:
        csd_cur_t, csdord_t = ugf.tfr2csd(tfrres_cur_, sfreq, returnOrder=1,
                                        ind_pairs=None,
                                        parcel_couplings=parcel_couplings_t,
                                        LFP2LFP_couplings=LFP2LFP_couplings_t,
                                        LFP2parcel_couplings=LFP2parcel_couplings_t,
                                        oldchns=chnames_tfr,
                                        newchns=newchns,
                                        res_group_id=newchn_grouping_ind,
                                        log=log_during_csd)


In [ ]:
dd.shape, csd_pri[0].shape

## CSD

In [ ]:
freqs[:10], freqs[30:]

In [ ]:
from plots import *
plt.close('all')

#dd = np.abs( csd_cur_t[0] )
dd = np.abs( csd_pri[0][1] )

plt.figure()
ax = plt.gca(); axind += 1
print(np.max(dd ), dd.shape)
#for feati in range(30, dd.shape[0] ):
for feati in range(8, dd.shape[0] ):
    d = dd[feati] 
    ax.plot(  d) 

ax.plot( dat_pri[0][0][::skip] / 30  )
ax.set_title('LFP higher_freq')
    
plt.figure()
ax = plt.gca();
for feati in range(8):
    d = dd[feati] 
    ax.plot(  d) 
#         if np.max(d) > 1e-10:
#             print(fi)
ax.plot( dat_pri[0][0][::skip] / 10  )
shadeAnn(ax,anns_artif, 0,0.1 ,plot_bins = 1, sfreq=sfreq, skip=skip)
#ax.fill_betweenx([0,.01],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.set_title('LFP lower_freq')

dd = np.abs( csd_LFP_HFO_pri[0][0] )
print(csd_LFP_HFO_pri[0].shape)
plt.figure()
ax = plt.gca();
for feati in range(27):
    d = dd[feati] 
    ax.plot(  d) 
#         if np.max(d) > 1e-10:
#             print(fi)
ax.plot( dat_pri[0][0][::skip] / 100  )

shadeAnn(ax,anns_artif, 0,0.01 ,plot_bins = 1, sfreq=sfreq, skip=skip)
#ax.fill_betweenx([0,.01],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.set_title('HFO lower_freq')

In [ ]:
ax = plt.gca(); axind += 1
dd = csd_pri[0]
for chi in range(2):
    for fi in range(30,dd.shape[1] ):
        ax.plot( np.abs( dd[chi][fi] ) )
        
ax.plot( dat_pri[0][0][::skip] / 30  )
ax.fill_betweenx([0,.15],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.set_title('csd_pri LFP higher_freq')

In [ ]:
bpow_abscsd_pri, bpow_imagcsd, csdord_strs_pri, csdord_strs_HFO_pri,bpow_abscsd_LFP_HFO_pri  = \
    ugf.bandAverage( freqs,freqs_inc_HFO,csd_pri,csdord_pri,csdord_LFP_HFO_pri,
            csd_LFP_HFO_pri, fbands,fband_names, fband_names_inc_HFO,
            newchns, subfeature_order_lfp_hires, log_before_bandaver= log_before_bandaver )


In [ ]:
plt.plot(freqs,n_cycles)

In [ ]:
freqs,n_cycles, Ws, windowsz_max = utils.prepFreqs(min_freq = min_freq, max_freq = 90,
                                                   frmults=[2*m,m,m], sfreq=sfreq )


In [ ]:
freqs

In [ ]:
len(Ws), len(Ws[0]),len(freqs)

### Inspect wavelets

In [ ]:
for ww in Ws:
    lens = []
    for w in ww:
        lens += [len(w)]
    print( list( zip(freqs,lens) ) )


In [ ]:
print( 'bpow_abscsd_pri[0].shape =', bpow_abscsd_pri[0].shape )
print( 'bpow_abscds_all_reshaped.shape =' ,bpow_abscds_all_reshaped.shape )
print( 'bpow_abscsd_LFP_HFO_pri[0].shape = ', bpow_abscsd_LFP_HFO_pri[0].shape )

# abscsd -- bandpow-averaged
plt.figure(figsize=(15,3))
ax = plt.gca(); axind += 1
chi = 3
dd = bpow_abscsd_pri[0][chi]
#dd = bpow_abscsd_LFP_HFO_pri[0][0]
print(dd.shape)
#dd = bpow_abscds_all_reshaped
for feati in range(0,dd.shape[0]):
    d = np.abs( dd[feati] )
    ax.plot( d, label=f'{chi}:{feati}' )
    if np.max(d) > 1:
        print(feati,feat_dict['con']['names'][feati])
        
dd1 = bpow_abscsd_LFP_HFO_pri[0][0]
print(dd.shape)
#dd = bpow_abscds_all_reshaped
for feati in range(0,dd1.shape[0]):
    d = np.abs( dd1[feati] ) 
    d = d / np.max(d) * np.max(dd)
    ax.plot( d, label=f'{feati} HFO' )
    if np.max(d) > 1:
        print(feati,feat_dict['con']['names'][feati])

ax.plot( dat_pri[0][0][::skip] * np.max(dd) * 1.1  , ls=':')
ax.set_title('abscsd higher_freq')
ax.fill_betweenx([0,np.max(dd)],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
ax.legend()


In [ ]:
raw_perband_flt_pri[0].keys()

In [ ]:
    raw_perband_flt_pri, raw_perband_bp_pri, chnames_perband_flt_pri, chnames_perband_bp_pri  = \
        ugf.bandFilter(rawnames, times_pri, main_sides_pri, side_switched_pri,
              sfreqs, skips, dat_pri_persfreq, fband_names_inc_HFO, gv.fband_names_HFO_all,
              fbands, n_jobs_flt, allow_CUDA, subfeature_order, subfeature_order_lfp_hires,
              smoothen_bandpow, ann_MEGartif_prefix_to_use, artif_handling='reject')


In [ ]:
plt.figure(figsize=(16,3))
ax = plt.gca()

for bn,r in raw_perband_flt_pri[0].items():
    d = r.get_data()
    print(bn, np.where( np.abs(d[:,-1] ) > 1e-10 )[0] )
    ax.plot(d.T , label=bn)
    
#raw_perband_bp_pri

ax.fill_betweenx([-0.6,.6],  (offset_start-windowsz),  offset_start, color='red', alpha=0.15)
ax.plot( dat_pri[0][0] * 0.7 , ls=':' )
#ax.legend(loc='lower left')

In [ ]:
len(dat_pri_persfreq[2])

In [ ]:
sfreqs

In [ ]:
dat_pri_persfreq

In [ ]:
ax = plt.gca()
ax.plot(dat_pri_persfreq[1][0].T)

In [ ]:
main_side_let

In [ ]:
# if artifact is in the beginning or end, filtering causes artifact

In [ ]:
# l_trans_bandwidth float | str

#     Width of the transition band at the low cut-off frequency in Hz (high pass or cutoff 1 in bandpass). Can be “auto” (default) to use a multiple of l_freq:
#     min(max(l_freq * 0.25, 2), l_freq)

#     Only used for method='fir'.
# h_trans_bandwidth float | str

#     Width of the transition band at the high cut-off frequency in Hz (low pass or cutoff 2 in bandpass). Can be “auto” (default in 0.14) to use a multiple of h_freq:

#     min(max(h_freq * 0.25, 2.), info['sfreq'] / 2. - h_freq)


In [ ]:
plt.figure(figsize=(15,3))
r = utils.makeSimpleRaw(dat_pri_persfreq[0][0],copy=1)
ann_ = mne.Annotations([0.2],[0.7],['BAD_R'])
r.set_annotations(ann_)
plt.axvline(ann_.onset[0] * sfreq)
plt.axvline((ann_.onset[0] + ann_.duration[0]) * sfreq)
r0 = r.copy()
r0.filter(l_freq=13,h_freq=16, pad='symmetric', skip_by_annotation=f'BAD_R', phase='zero')
plt.plot( r0.get_data()[0], label='>1s zero' )

r1 = r.copy()
r1.filter(l_freq=13,h_freq=16, pad='symmetric', skip_by_annotation=f'BAD_R',filter_length='1s',
          l_trans_bandwidth=2, h_trans_bandwidth=2, phase='minimum')
plt.plot( r1.get_data()[0], label='1s' )

r2 = r.copy()
r2.filter(l_freq=13,h_freq=16, pad='symmetric', skip_by_annotation=f'BAD_R',filter_length='2s',
          l_trans_bandwidth=2, h_trans_bandwidth=2, phase='minimum')
plt.plot( r2.get_data()[0], label='2s' )

r3 = r.copy()
r3.filter(l_freq=13,h_freq=16, pad='symmetric', skip_by_annotation=f'BAD_R',filter_length='2s',
          l_trans_bandwidth=2, h_trans_bandwidth=2, phase='zero')
plt.plot( r3.get_data()[0], label='2s' )

# r2 = r.copy()
# r2.filter(l_freq=3,h_freq=6, skip_by_annotation=f'BAD_R')
# plt.plot( r2.get_data()[0] )

plt.plot( dat_pri[0][0] * 1 , ls=':' )

plt.legend()

In [ ]:
r1 = r.copy()
r1.filter(l_freq=3,h_freq=6, pad='symmetric', skip_by_annotation=f'BAD_R', len)
plt.plot( r1.get_data()[0] )

In [ ]:
ax = plt.gca()
#for dd in dat_pri_persfreq:
ax.plot(dat_pri_persfreq[0][0].T)


In [ ]:
parcel_couplings

In [ ]:
parcel_couplings_t = {(4, 4): [(2, 2)]}
parcel_couplings_t, LFP2parcel_couplings_t, LFP2LFP_couplings_t

In [ ]:
means_perband

In [ ]:
rbcorr_use_local_means_t = 0

In [ ]:
means_perband_flt_pri[0]

In [ ]:
import copy
means_perband_t = copy.deepcopy( means_perband_flt_pri[0])
for k in means_perband_t:
    means_perband_t[k] = np.zeros( means_perband_t[k].shape)

In [ ]:
if 'rbcorr' in features_to_use:  #raw band corr
    if bands_only == 'fine':
        bandPairs = [('tremor','tremor', 'corr'),
                    ('low_beta','low_beta', 'corr'), ('high_beta','high_beta', 'corr'),
                    ('low_gamma','low_gamma', 'corr') , ('high_gamma','high_gamma', 'corr') ]
    else:
        bandPairs = [('tremor','tremor', 'corr'), ('beta','beta', 'corr'), ('gamma','gamma', 'corr') ]

    rbcorrs_pri     = [0] * len(rawnames)
    wbd_rbcorr_pri  = [0] * len(rawnames)
    rbcor_names_pri = [0] * len(rawnames)
    for rawind in range(len(dat_pri)):
        fn = pre_rbcorr_fname.format(rawnames[rawind], n_channels_pri[rawind] )
        fn = pjoin(gv.data_dir, output_subdir, fn)
        if load_rbcorr and os.path.exists(fn):
            print('Load rbcorr from {}'.format(fn) )
            f = np.load(fn, allow_pickle=True)
            bpcorrs      =f['rbcorrs'    ]
            bpcor_names  =f['rbcor_names']
            wbd_bpcorr   =f['wbd_rbcorr' ]
        else:
            print('Starting rbcorr for datset {}'.format(rawind) )
            raw_perband =  raw_perband_flt_pri[rawind]
            chnames_perband =  chnames_perband_flt_pri[rawind]
            if rbcorr_use_local_means:
                means_perband = None
            else:
                means_perband = means_perband_flt_pri[rawind]

            rbcorrs = []
            rbcor_names = []
            for bp in bandPairs:
                print('band pair ',bp)
                rbcorrs_curbp,rbcor_names_curbp,dct_nums, wbd_rbcorr = \
                computeCorr(raw_perband, names=chnames_perband,
                                        defnames=chnames_tfr,
                                        parcel_couplings=parcel_couplings_t,
                                        LFP2parcel_couplings=LFP2parcel_couplings_t,
                                        LFP2LFP_couplings=LFP2LFP_couplings_t,
                                        res_group_id=newchn_grouping_ind,
                                        skip=skip, windowsz = windowsz,
                                        band_pairs = [bp], n_jobs=n_jobs,
                                        positive=0, templ='{}_.*',
                                        roi_labels=roi_labels,
                                        sort_keys=srcgrouping_names_sorted,
                                        means=means_perband_t,
                                        local_means=rbcorr_use_local_means_t, pad='pre_left')
                assert len(rbcor_names_curbp) > 0
                gc.collect()
                rbcorrs     += rbcorrs_curbp
                rbcor_names += rbcor_names_curbp

            rbcorrs = np.vstack(rbcorrs)

            for feati in range(len(rbcor_names) ):
                rbcor_names[feati] = 'rb' + rbcor_names[feati]

            if save_rbcorr:
                print('Saving rbcorr ',fn)
                #fn = pjoin(gv.data_dir, output_subdir,'{}_rbcorr.npz'.format(rawnames[rawind] ) )
                np.savez( fn , rbcorrs=rbcorrs, rbcor_names=rbcor_names, wbd_rbcorr=wbd_rbcorr)
                gc.collect()

        rbcor_names_pri[rawind] = rbcor_names
        rbcorrs_pri    [rawind] = rbcorrs
        wbd_rbcorr_pri [rawind] = wbd_rbcorr
    if do_cleanup:
        del  raw_perband_flt_pri
        del  rbcorrs
    gc.collect()


In [ ]:
plt.figure(figsize=(15,3))
ax = plt.gca()
d = bpcorrs_pri[0][0]
ax.plot( d, label='bp'  )

d = rbcorrs_pri[0][0]
ax.plot( d, label='rb'  )



print(bpow_abscds_all_reshaped.shape)
d = bpow_abscds_all_reshaped[2]

ax.plot( d / 5, label='abscsd'  )
ax.fill_betweenx([0,.02],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)

ax.set_title('bpcorr and rbcorr')

plt.plot( dat_pri[0][0][::skip] * 0.04 , ls=':' )
ax.legend()

In [ ]:
wbd_rbcorr

In [ ]:
(offset_start-windowsz)//skip

In [ ]:
plt.plot( dat_pri[0][0][::skip] * 0.04 , ls=':' )

In [ ]:
help(r.apply_hilbert)

In [ ]:
help( r.filter )

In [ ]:
r.ch_names

In [ ]:
mne.pick_channels_regexp(subfeature_order,'msrc.*')

In [ ]:
anns_artif

In [ ]:
plt.close('all')

In [ ]:
plt.plot(dat_lfp_hires_pri[0].T)

In [ ]:
from plots import *
plt.suptitle('Filtered')
plt.figure(figsize=(15,6))
plt.plot( dat_pri[0][0] * 0.02 , ls=':' )
plotStepsWbd(rbcorrs_pri[0][0], wbd_rbcorr, sfreq)

ax = plt.gca()
for bn,r in raw_perband_flt_pri[0].items():
    d = r.get_data()
    #print(bn, np.where( np.abs(d[:,-1] ) > 1e-10 )[0] )
    ax.plot(d.T / 30 + 0.05 , label=bn)

    

plt.figure(figsize=(15,6))
plt.suptitle('Bandpower')
plt.plot( dat_pri[0][0] * 0.02 , ls=':' )
plotStepsWbd(bpcorrs_pri[0][0], wbd_rbcorr, sfreq)

ax = plt.gca()
for bn,r in raw_perband_bp_pri[0].items():
    if bn != 'HFO':
        continue
    d = r.get_data()
    xs =  np.arange(dat_pri[0].shape[-1] )
    if r.info['sfreq']  > 256:
        xs =  np.arange(len(dat_lfp_hires_pri[0].shape[-1]) / sfreq_hires * sfreq )
    #print(bn, np.where( np.abs(d[:,-1] ) > 1e-10 )[0] )
    ax.plot(xs,  d[:2].T / 30 + 0.05 , label=f'LFP  {bn}')
    if d.shape[0] > 2:
        ax.plot(xs,d[2:].T / 30 + 0.025, label=f'msrc {bn}' )

ax.legend()

In [ ]:
d.shape

In [ ]:
anns_artif

In [ ]:


# TODO: test subfeature_order
res = imputeInterpArtif(dat_pri[0].T, anns_artif, subfeature_order)
res = res.T

In [ ]:
r.annotations

In [ ]:
help(r.get_data)

In [ ]:
res.shape

In [ ]:
plt.figure(figsize=(15,3))
ax = plt.gca()
ax.plot( res[0] )
ax.plot( res[-1] * 1.5)
#shadeAnn(ax, anns_artif, 0, 1.5, skip=1, plot_bins=1)
ax.legend()

In [ ]:
import pandas as pd
dat_pd_t = np.zeros( ( 30, 2))
dat_pd_t[:,0] = np.arange(df.shape[0])
dat_pd_t[6:10] = np.nan
df = pd.DataFrame(dat_pd_t)
#df_interpol = df.groupby('house')\
#                 .resample('D')\
#                 .mean()
df_interpol = df.copy()
df_interpol[0] = df_interpol[0].interpolate()
#df_interpol.head(4)

In [ ]:
df.get()

In [ ]:
df.interpolate(axis=0)

In [ ]:
help( df.interpolate )

In [ ]:
df.shape

In [ ]:
gc.collect()

In [ ]:
df.shape

In [ ]:
df_interpol

In [ ]:
df[0,:] = np.arange(df.shape[0])

In [ ]:
csdord_strs

In [ ]:
d = bpcorrs_pri[0][0]
plt.plot( d  )

d = rbcorrs_pri[0][0]
plt.plot( d  )



In [ ]:
plt.plot(dat_for_tfr.T)

In [ ]:
#help(plt.figure)

In [ ]:
offset_sec

In [ ]:
dat_p

In [ ]:
dat_pri[0].shape

In [ ]:
sfreq * 

In [ ]:
dat_for_tfr.shape

In [ ]:
dat_lfp_hires_pri[0].shape

In [ ]:
#dat_for_tfr_t= np.zeros(dat_lfp_hires_pri[0].shape)
dat_for_tfr_t = np.zeros( (2,sfreq_hires * 10 ) )
offset_sec_t = 3
offset2_sec_t = 6
xs = np.arange(dat_for_tfr_t.shape[1] - offset_sec_t * sfreq_hires)  / sfreq_hires
#dat_for_tfr_t[0,offset_sec_t*sfreq_hires : ] = np.sin( xs * 2 * 5* np.pi )
dat_for_tfr_t[0,offset_sec_t*sfreq_hires : ] = 1
#dat_for_tfr_t[0,offset2_sec_t * sfreq_hires:] = 0

plt.figure(figsize=(14,3))
plt.plot(dat_for_tfr_t.T)

In [ ]:
dat_for_tfr_t.shape[1] / skip

In [ ]:
# calc HFO once again just in case
tfrres_LFP_t,wbd_HFO = utils.tfr(dat_for_tfr_t, sfreq_hires, freqs_inc_HFO, n_cycles_inc_HFO,
                    windowsz_hires, decim = skip_hires // skip_div_TFR, n_jobs=n_jobs_tfr, mode='untouched')
tfrres_LFP_LFO_t = tfrres_LFP_t[:,:len(freqs),:]
tfrres_LFP_HFO_t = tfrres_LFP_t[:,len(freqs):,:]



# plot LFP HFO
fig,axs = plt.subplots(3,1,figsize=(12,6))
ax=axs[0]
ax.set_title('lowfreq_LFP_HFO')
for chi in range(2):
    for fi in range(27,tfrres_LFP_LFO_t.shape[1] ):
        ax.plot( np.abs( tfrres_LFP_LFO_t[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] / 8  )
ax=axs[1]
for chi in range(2):
    for fi in range(27 ):
        ax.plot( np.abs( tfrres_LFP_LFO_t[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] *2  )

ax=axs[2]
for chi in range(2):
    for fi in range(tfrres_LFP_HFO_t.shape[1] ):
        ax.plot( np.abs( tfrres_LFP_HFO_t[chi][fi] ) )
ax.plot( dat_pri[0][0][::skip] / 8  )

In [ ]:
tfrres_wbd_pri[0]

In [ ]:
d1 = act_pri[0][0]
d2 = mob_pri[0][0]
d3 = compl_pri[0][0]
plt.close('all')
plt.plot( d1 / np.max(d1)  ,label='act', marker='*' )
plt.plot( d2 / np.max(d2), label='mob' )
plt.plot( d3 / np.max(d3)  , label='compl' )


plt.plot( dat_pri[0][0][::skip] *2  )
plt.fill_betweenx([0,2],  (offset_start-windowsz)//skip,  offset_start//skip, color='red', alpha=0.15)
plt.legend()

In [ ]:
offset_start - windowsz

In [ ]:
n = int(windowsz*2 + len(tt[0]) - nb - windowsz/2)
print(n, n/windowsz )
wbd_t[1,n]

In [ ]:
np.arange( tt.shape[-1] - windowsz*4)

In [ ]:
strt = (tt.shape[-1] + windowsz -1 )//2; print(strt)
wbd_new_st = np.arange(strt, )

In [ ]:
# if I sartt from 1.5 *wsz, then  tfr (mode = valid) starts from 0
# if I sartt from 1 *wsz, then  tfr (mode = untouched) starts from 0

In [ ]:
windowsz / decim * 2

In [ ]:
np.max(tt)

In [ ]:
r = mne.time_frequency.tfr_array_morlet(tt[None,:],sfreq,freqs,n_cycles,n_jobs=1,decim=32)

r.shape

np.max(np.abs(r))

In [ ]:
fr = 12 
#m1 = 1.5
m1 = 2; m2 = 9; m3 = 11
tt = np.zeros((2,windowsz * m3 )  )
sz = int(windowsz*(m2-m1) )
tt[0,int(windowsz*m1):int(windowsz*m2)] = 1 * np.sin(fr * 2 * np.pi * np.arange( sz) / sz)
tt[0,windowsz*m2:] = 0

decim = 64
#decim = 1
tfrres_t,wbd_t = utils.tfr(tt, sfreq, freqs, n_cycles,
                        windowsz, decim = decim,
                        n_jobs=n_jobs_tfr, mode='valid')

change_mode_freqi = 27
for freqSlice in [slice(None,change_mode_freqi), slice(change_mode_freqi,None)]:
    plt.figure(figsize=(16,4))

    xs = np.arange(0,len(tt[0]),1)
    # before around 32 slower freq use larger windows!
    for chi in range(2):
        for fi in np.arange(tfrres_t.shape[1]  )[freqSlice]:
            d = tfrres_t[chi][fi]
            plt.plot( xs[::decim][:len(d) ], 1 + 50 * np.abs( d ) )

    nb = tfrres_t[chi][fi].shape[0]
    plt.plot(xs, tt[0] , lw=1, label='orig')

    #ycur = tt[0][len(tt[0]) - nb*decim:] / 2
    #plt.plot(xs[:len(ycur)], ycur , lw=3, ls=':', label='shift len dif')
    #plt.axvline(x=windowsz*2 + len(tt[0]) - nb - windowsz/2, ls=':')
    #plt.axvline(x=windowsz*3 + len(tt[0]) - nb + windowsz/2, ls=':')
    for x in np.arange(0,len(tt[0]),windowsz):
        plt.axvline(x=x, ls=':',c='blue')
    bi = 0
    b0 = wbd_t[0,bi]
    b1 = wbd_t[1,bi]
    plt.fill_betweenx([-1,1],b0,b1, ls=':',color='red',alpha=0.2)
    #ei = tfrres_t.shape[-1]- windowsz *2 // decim 
    ei = -1
    e0 = wbd_t[0,ei]
    e1 = wbd_t[1,ei]
    plt.fill_betweenx([-1,1],e0,e1, ls=':',color='red',alpha=0.2)
    plt.legend()


#assert first and last window get zeros and any shifts get a bit of stuff
#plt.xlim(800,1600)

In [ ]:
(windowsz // decim)

In [ ]:
tt.shape

In [ ]:
b0,b1

In [ ]:
post_start_segment

In [ ]:
len( Ws[0] ), len(Ws)

In [ ]:
Ws[0][26].shape

In [ ]:
ei

In [ ]:
tfrres_t.shape[-1]

In [ ]:
wbd_t.shape

In [ ]:
wbd_t[1]

In [ ]:
dat_for_tfr.shape, dat_pri[0].shape

In [ ]:
plt.figure()
plt.plot(dat_pri[0][0])

In [ ]:
X_pri[0].shape

In [ ]:
wbd

In [ ]:
plot_types

In [ ]:
do_plot_feat_stat_scatter

In [ ]:
ivalis

In [ ]:
gv.CUDA_state

In [ ]:
n_jobs_flt

In [ ]:
tfr_std

In [ ]:
tfrres_LFP_LFO.shape

In [ ]:
tfrres.shape

In [ ]:
len(Ws)

In [ ]:
[len(w) for w in Ws[2] ]

In [ ]:
[len(w) for w in Ws[0] ]

In [ ]:
# %load ~/soft/_pyenvs/obddp/lib/python3.8/site-packages/mne/time_frequency/tfr.py